<H1>The battle of the neighborhoods - Week 2-2</h1>

<h2>A link to your Notebook on your Github repository pushed showing your code.</h2>

In [1]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-base-1.15.4          |   py36h81de0dd_0         4.2 MB  anaconda
    numpy-1.15.4               |   py36h1d66e8a_0          35 KB  anaconda
    beautifulsoup4-4.8.1       |           py36_0         153 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    soupsieve-1.9.5            |           py36_0          61 KB  anaconda
    mkl_fft-1.0.6              |   py36h7dd41cf_0         150 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  an

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0         conda-forge
    geopy:         1.20.0-py_0       conda-forge

The following pa

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [5]:
website_url = requests.get("https://es.wikipedia.org/wiki/Anexo:Localidades_de_Bogot%C3%A1").text

In [6]:
soup = BeautifulSoup(website_url,'html5lib')
My_table = soup.find('table',{'class':'sortable wikitable'})
My_table
links = My_table.findAll('a')
links
Localidades = []
for link in links:
    Localidades.append(link.get('title'))
    
print(Localidades)
del Localidades[0:2]
Localidades

[None, None, 'Usaquén', 'Chapinero', 'Santa Fe (Bogotá)', 'San Cristóbal (Bogotá)', 'Usme', 'Tunjuelito', 'Bosa (Bogotá)', 'Kennedy (Bogotá)', 'Fontibón', 'Engativá', 'Suba', 'Barrios Unidos (Bogotá)', 'Teusaquillo', 'Los Mártires', 'Antonio Nariño (Bogotá)', 'Puente Aranda', 'La Candelaria', 'Rafael Uribe Uribe (Bogotá)', 'Ciudad Bolívar (Bogotá)', 'Sumapaz (Bogotá)']


['Usaquén',
 'Chapinero',
 'Santa Fe (Bogotá)',
 'San Cristóbal (Bogotá)',
 'Usme',
 'Tunjuelito',
 'Bosa (Bogotá)',
 'Kennedy (Bogotá)',
 'Fontibón',
 'Engativá',
 'Suba',
 'Barrios Unidos (Bogotá)',
 'Teusaquillo',
 'Los Mártires',
 'Antonio Nariño (Bogotá)',
 'Puente Aranda',
 'La Candelaria',
 'Rafael Uribe Uribe (Bogotá)',
 'Ciudad Bolívar (Bogotá)',
 'Sumapaz (Bogotá)']

In [7]:
df = pd.DataFrame()
df['Localidades'] = Localidades
df

,Localidades
0,Usaquén
1,Chapinero
2,Santa Fe (Bogotá)
3,San Cristóbal (Bogotá)
4,Usme
5,Tunjuelito
6,Bosa (Bogotá)
7,Kennedy (Bogotá)
8,Fontibón
9,Engativá


In [8]:
def get_coords_local(localidad, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name
    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]
    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             localidad+', Bogota, Bogota Capital District',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [9]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(localidad=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln
df2

Usaquén
Chapinero
Santa Fe (Bogotá)
San Cristóbal (Bogotá)
Usme
Tunjuelito
Bosa (Bogotá)
Kennedy (Bogotá)
Fontibón
Engativá
Suba
Barrios Unidos (Bogotá)
Teusaquillo
Los Mártires
Antonio Nariño (Bogotá)
Puente Aranda
La Candelaria
Rafael Uribe Uribe (Bogotá)
Ciudad Bolívar (Bogotá)
Sumapaz (Bogotá)


,Localidades,Latitude,Longitude
0,Usaquén,4.694969,-74.031093
1,Chapinero,4.645377,-74.061943
2,Santa Fe (Bogotá),4.602204,-74.078837
3,San Cristóbal (Bogotá),4.548658,-74.047473
4,Usme,4.508155,-74.114328
5,Tunjuelito,4.562204,-74.127647
6,Bosa (Bogotá),4.609913,-74.184710
7,Kennedy (Bogotá),4.631582,-74.151319
8,Fontibón,4.673327,-74.144732
9,Engativá,4.708695,-74.109643


In [10]:
address = 'Bogotá, Colombia'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Bogotá are {}, {}.'.format(latitude, longitude))
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

The decimal coordinates of Bogotá are 4.59808, -74.0760439.


,Localidades,Latitude,Longitude
0,Usaquén,4.694969,-74.031093
1,Chapinero,4.645377,-74.061943
2,Santa Fe (Bogotá),4.602204,-74.078837
3,San Cristóbal (Bogotá),4.548658,-74.047473
4,Usme,4.508155,-74.114328
5,Tunjuelito,4.562204,-74.127647
6,Bosa (Bogotá),4.609913,-74.184710
7,Kennedy (Bogotá),4.631582,-74.151319
8,Fontibón,4.673327,-74.144732
9,Engativá,4.708695,-74.109643


In [11]:
# create map of Bogotá using latitude and longitude values
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bogota)  
    
map_bogota

In [12]:
# function to repeat the exploring process to all the neighborhoods in Toronto
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Localidad', 
                  'Localidad Latitude', 
                  'Localidad Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [13]:
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radiu
CLIENT_ID = 'UZVWVNFW1EYCXTWMP3BNUTXYXFC1ZCIXWPF4440JVP4OKNP3'
CLIENT_SECRET = 'LYZXKMREOJOQ1A2QYLW1O25IJ1RCUOT0GD4IJEKJVYNMR2S5'
VERSION = '20181020'

In [14]:
# Use category id 4bf58dd8d48988d14e941735 to only get the American Restaurant
bogota_venues_Restaurant = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d14e941735')
bogota_venues_Restaurant.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Mister Ribs,4.695403,-74.030600,American Restaurant
1,Usaquén,4.694969,-74.031093,La Hamburguesería,4.695310,-74.031716,Burger Joint
2,Usaquén,4.694969,-74.031093,Madison,4.695443,-74.030262,Burger Joint
3,Usaquén,4.694969,-74.031093,Rock 'N Stop,4.694666,-74.032456,American Restaurant
4,Usaquén,4.694969,-74.031093,Enfuego,4.694105,-74.031857,New American Restaurant


In [15]:
bogota_venues_Restaurant.shape

(74, 7)

In [16]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Localidad'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [17]:
map_bogota_Restaurant = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_Restaurant, 'red', map_bogota_Restaurant)
map_bogota_Restaurant

In [18]:
# Use category id 56aa371be4b08b9a8d573517 to only get the Bussiness Center
bogota_venues_BusinessCenter = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='56aa371be4b08b9a8d573517')
bogota_venues_BusinessCenter.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Flormorado,4.694266,-74.032539,Business Center
1,Usaquén,4.694969,-74.031093,Usaquén Plaza,4.696983,-74.031150,Shopping Mall
2,Usaquén,4.694969,-74.031093,Kaiwa Centro Empresarial,4.698918,-74.030767,Business Center
3,Usaquén,4.694969,-74.031093,Edwards Lifesciences Colombia,4.692951,-74.033920,Business Center
4,Usaquén,4.694969,-74.031093,Ignition Agencia BTL,4.699775,-74.032768,Business Center


In [19]:
bogota_venues_BusinessCenter.shape

(27, 7)

In [20]:
map_bogota_venues_BusinessCenter= folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_BusinessCenter,'green', map_bogota_venues_BusinessCenter)
map_bogota_venues_BusinessCenter


In [21]:
# Use category id 4bf58dd8d48988d1ff931735 to only get the Convention Center
bogota_venues_concenter = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ff931735')
bogota_venues_concenter.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Centro Empresarial Paseo Real,4.697782,-74.029324,Convention Center
1,Usaquén,4.694969,-74.031093,AR,4.692235,-74.035281,Convention Center
2,Usaquén,4.694969,-74.031093,AR-centro de convenciones,4.692206,-74.035335,Convention Center
3,Usaquén,4.694969,-74.031093,Villa Santa Maria De Usaquen,4.695387,-74.031345,Meeting Room
4,Usaquén,4.694969,-74.031093,1ra Convención Crowdfunding Prodigy Network,4.693362,-74.033349,Convention Center


In [88]:
bogota_venues_concenter.shape

(41, 7)

In [89]:
map_bogota_concenter = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_concenter, 'gold', map_bogota_concenter)
map_bogota_concenter

In [90]:
# Use category id 4d4b7105d754a06375d81259 to only get the office
bogota_venues_office = getNearbyVenues(names=df3['Localidades'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
bogota_venues_office.head()

,Localidad,Localidad Latitude,Localidad Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Usaquén,4.694969,-74.031093,Centro Médico de la Sabana,4.696026,-74.031596,Doctor's Office
1,Usaquén,4.694969,-74.031093,WeWork Usaquén,4.694304,-74.032745,Coworking Space
2,Usaquén,4.694969,-74.031093,Bistronomy,4.695679,-74.030267,French Restaurant
3,Usaquén,4.694969,-74.031093,Santa Ana Medical Center,4.696085,-74.032065,Doctor's Office
4,Usaquén,4.694969,-74.031093,Cisco Systems Colombia,4.692935,-74.034419,Office


In [91]:
bogota_venues_office.shape

(793, 7)

In [92]:
map_bogota_office = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(bogota_venues_office, 'fuchsia', map_bogota_office)
map_bogota_office

In [93]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Localidad').count()
    
    for n in startDf['Localidad']:
        try:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Localidad'] == n,columnTitle] = 0

In [94]:
df_data = df3.copy()
df_data.rename(columns={'Localidades':'Localidad'}, inplace=True)
addColumn(df_data, 'Restaurant', bogota_venues_Restaurant)
addColumn(df_data, 'Business Center', bogota_venues_BusinessCenter)
addColumn(df_data, 'Convention Center', bogota_venues_concenter)
addColumn(df_data, 'Offices', bogota_venues_office)
df_data

,Localidad,Latitude,Longitude,Restaurant,Business Center,Convention Center,Offices
0,Usaquén,4.694969,-74.031093,12.0,8.0,8.0,50.0
1,Chapinero,4.645377,-74.061943,15.0,6.0,5.0,50.0
2,Santa Fe (Bogotá),4.602204,-74.078837,15.0,1.0,6.0,50.0
3,San Cristóbal (Bogotá),4.548658,-74.047473,0.0,0.0,0.0,0.0
4,Usme,4.508155,-74.114328,0.0,0.0,0.0,31.0
5,Tunjuelito,4.562204,-74.127647,0.0,0.0,0.0,44.0
6,Bosa (Bogotá),4.609913,-74.184710,0.0,0.0,2.0,39.0
7,Kennedy (Bogotá),4.631582,-74.151319,3.0,0.0,1.0,43.0
8,Fontibón,4.673327,-74.144732,1.0,3.0,2.0,44.0
9,Engativá,4.708695,-74.109643,2.0,0.0,1.0,48.0


In [96]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_Restaurant = -1

# positive weight, because high school students are good customers
weight_BusinessCenter = 1

# positive weight, because uni students are good customers
weight_concenter = 1.5

# positive weight because employees are even better customers
weight_offices = 2

In [97]:
df_weighted = df_data[['Localidad']].copy()

In [99]:
df_weighted['Score'] = df_data['Restaurant'] * weight_Restaurant + df_data['Business Center'] * weight_BusinessCenter + df_data['Convention Center'] * weight_concenter + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Localidad,Score
0,Usaquén,108.0
11,Barrios Unidos (Bogotá),102.5
12,Teusaquillo,99.0
1,Chapinero,98.5
15,Puente Aranda,97.0
9,Engativá,95.5
2,Santa Fe (Bogotá),95.0
14,Antonio Nariño (Bogotá),94.0
8,Fontibón,93.0
16,La Candelaria,92.5


In [101]:
map_bog_result = folium.Map(location=[latitude, longitude], zoom_start=15)

bog_win = df3[df3['Localidades'] == 'La Candelaria']

for lat, lng, local in zip(bog_win['Latitude'], bog_win['Longitude'], bog_win['Localidades']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_bog_result) 

addToMap(bogota_venues_Restaurant[bogota_venues_Restaurant['Localidad'] == 'Usaquén'], 'red', map_bog_result)
addToMap(bogota_venues_BusinessCenter[bogota_venues_BusinessCenter['Localidad'] == 'Usaquén'], 'green', map_bog_result)
addToMap(bogota_venues_concenter[bogota_venues_concenter['Localidad'] == 'Usaquén'], 'gold', map_bog_result)
addToMap(bogota_venues_office[bogota_venues_office['Localidad'] == 'Usaquén'], 'fuchsia', map_bog_result)

map_bog_result